# AI Mathematical Olympiad - Progress Prize 3 Solution

Complete solution using AGI engine breeding, training, and optimization for olympiad-level math problems.

**Key Features:**
- Engine breeding from top Gen-4 AGI engines
- Extended training on all math domains
- Chain-of-thought reasoning
- Hyperparameter optimization
- Ensemble methods with weighted voting

**Target:** 100% accuracy on reference problems

## Setup and Configuration

In [ ]:
import os
import sys
import logging

# Clone repository with pre-trained engines and submodules
# Note: In Kaggle environment, use datasets or pre-uploaded code
REPO_URL = "https://github.com/pythpythpython/kaggle-comps.git"

# For Kaggle: Clone the repository with submodules if not already present
# Uncomment and use your Kaggle Secret for GITHUB_PAT if repository is private:
# from kaggle_secrets import UserSecretsClient
# user_secrets = UserSecretsClient()
# GITHUB_PAT = user_secrets.get_secret("GITHUB_PAT")
# !git clone --recursive https://{GITHUB_PAT}@github.com/pythpythpython/kaggle-comps.git /kaggle/working/kaggle-comps

# For public repository (recommended):
# !git clone --recursive https://github.com/pythpythpython/kaggle-comps.git /kaggle/working/kaggle-comps

# Add solution to path
# Adjust path based on where the repository was cloned
sys.path.insert(0, '/kaggle/working/kaggle-comps/AI Mathematical Olympiad - Progress Prize 3')


In [ ]:
# Import solution components
from aimo3_solution.config import Config
from aimo3_solution.math_solver import MathSolver
from aimo3_solution.latex_parser import LaTeXParser
from aimo3_solution.answer_extractor import AnswerExtractor
from aimo3_solution.engine_breeder import EngineBreeder
from aimo3_solution.utils import setup_logging

# Initialize
logger = setup_logging(level=logging.INFO)
config = Config()

logger.info("AIMO3 Solution Initialized")

## Load Best Engine Configuration

In [ ]:
import json

def load_best_engine():
    """Load the best trained engine"""
    engine_path = os.path.join(config.trained_engines_dir, 'best_engine.json')
    
    if os.path.exists(engine_path):
        with open(engine_path, 'r') as f:
            return json.load(f)
    
    # Fallback: Create default engine
    logger.info("Creating default engine configuration")
    breeder = EngineBreeder(config)
    top_engines = breeder.select_top_engines(1)
    return breeder.create_engine_config(top_engines[0])

engine_config = load_best_engine()
logger.info(f"Using engine: {engine_config['name']}")
logger.info(f"Quality: {engine_config['quality']:.4f}")

## Initialize Solver

In [ ]:
# Initialize math solver
solver = MathSolver(config)
logger.info("Math solver initialized")

## Test on Reference Problems (Optional)

In [ ]:
# Optional: Test on reference problems to verify accuracy
from aimo3_solution.tester import Tester

tester = Tester(config)
accuracy, results = tester.test_engine_on_reference(engine_config, solver)

logger.info(f"Reference test accuracy: {accuracy:.2%}")

# Show some results
for result in results[:3]:
    logger.info(f"Problem {result.problem_id}: {'✓' if result.is_correct else '✗'}")

## Solve Test Problems using Kaggle API

In [ ]:
from kaggle_evaluation.aimo_3_inference_server import AIMO3InferenceServer

def predict(id: str, problem: str) -> int:
    """Predict function that solves a single problem.
    
    Args:
        id: Problem identifier
        problem: Problem text in LaTeX format
        
    Returns:
        Integer answer in range [0, 99999]
    """
    # Solve problem using the trained solver
    answer, reasoning = solver.solve(problem, engine_config)
    
    # Log progress periodically
    logger.info(f"Solved problem {id}: answer = {answer}")
    
    return answer

# Initialize inference server with the predict function
inference_server = AIMO3InferenceServer(predict)

logger.info("Starting inference server...")

# Start serving (this will call predict for each problem)
inference_server.serve()

logger.info("Inference server completed")

## Submission Note


In [ ]:
# Note: Submission file is automatically generated by the Kaggle inference server API
# The inference_server.serve() call above handles submission automatically
logger.info("Submission handled by Kaggle inference server API")

## Quality Check

In [ ]:
from aimo3_solution.quality_checker import QualityChecker

checker = QualityChecker(config)

# Validate answer format
format_passed, errors = checker.validate_answer_format(predictions)

logger.info(f"Format validation: {'✓ PASSED' if format_passed else '✗ FAILED'}")

if not format_passed:
    for error in errors[:5]:
        logger.error(error)

## Summary

In [ ]:
print("=" * 60)
print("AIMO3 SUBMISSION SUMMARY")
print("=" * 60)
print(f"Engine: {engine_config['name']}")
print(f"Quality: {engine_config['quality']:.4f}")
print(f"Problems solved: {len(predictions)}")
print(f"Submission file: {submission_path}")
print("=" * 60)